# Постановка задачи

Необходимо дообучить модель [”openai/whisper-tiny”](https://huggingface.co/openai/whisper-tiny) с использованием подмножества («en-US») датасета [”PolyAI/minds14”](https://huggingface.co/datasets/PolyAI/minds14).



*   Используйте первые **450 примеров для обучения**, остальные для оценки. Убедитесь, что вы установили `num_proc=1` при предварительной обработке набора данных с использованием метода `.map` (это обеспечит правильную отправку вашей модели для оценки).
*   Для оценки модели используйте метрики `wer` и `wer_ortho`. Однако *не* преобразуйте метрику в проценты, умножая на 100 (например, если WER равен 42%, мы ожидаем увидеть значение 0.42).

Для начала необходимо связать эту рабочую тетрадь с Hugging Face Hub, так как я планирую загрузить туда контрольную точку дообученной модели.

Для этого при запуске ячейки ниже необходимо ввести свой токен аутентификации на Hugging Face Hub. Найти свой токен аутентификации можно [здесь](https://huggingface.co/settings/tokens)

In [ ]:
!pip install huggingface_hub
!pip install datasets
!pip install git+https://github.com/huggingface/transformers
!pip install evaluate
!pip install jiwer
!pip install accelerate
!pip install gradio

In [2]:
from huggingface_hub import notebook_login

notebook_login()

# Импорт данных

Далее загрузим датасет и разобьем его на обучающую и тестовую выборки, согласно заданию (450 на обучение, оставшуюся часть на тест).

In [3]:
from datasets import load_dataset, DatasetDict

minds = DatasetDict()

minds["train"] = load_dataset(
    "PolyAI/minds14", name="en-US", split="train[0:450]"
)
minds["test"] = load_dataset(
    "PolyAI/minds14", name="en-US", split="train[450:]"
)

print(minds)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 450
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 113
    })
})


In [4]:
minds

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 450
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 113
    })
})

Выбранный датасет содержит дополнительную информацию, такую как: `'english_transcription'`, `'intent_class'`, `'lang_id'`. Эти признаки не потребуются для обучения ASR-модели. В связи с вышеизложенным мы можем избавиться от этих колонок.


In [5]:
minds = minds.select_columns(["audio", "transcription"])

В библиотеке 🤗 Transformers для модели Whisper существуют связанные экстрактор признаков и токенизатор, называемые
[WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor)
и [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer) соответственно.
Для упрощения работы с ними, эти два объекта объединены в одном классе, названном [WhisperProcessor](https://huggingface.co/docs/transformers/model_doc/whisper#transformers.WhisperProcessor).
Мы можем вызвать WhisperProcessor для выполнения как предварительной обработки звука, так и постобработки текстового токена.

Далее мы загрузим наш WhisperProcessor из предварительно обученной модели (аргументы `language=`, `task=` в обработчик не передаем, т.к. по умолчанию язык - английский, а задача - транскрибация):

In [6]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")

#Предварительная обработка данных

Для начала нужно убедиться что частота дискретизации входных экземпляров будет равна той, которую ожидает получить на вход наша модель (Whisper ожидает получать на вход файлы с частотой 16 кГц).

In [7]:
minds["train"].features

{'audio': Audio(sampling_rate=8000, mono=True, decode=True, id=None),
 'transcription': Value(dtype='string', id=None)}

Необходимо повысить `sampling_rate` до 16 кГц.

In [8]:
from datasets import Audio

sampling_rate = processor.feature_extractor.sampling_rate
minds = minds.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [9]:
minds["train"].features

{'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None),
 'transcription': Value(dtype='string', id=None)}

Отлично, `sampling_rate` = 16000

Теперь мы можем написать функцию для подготовки наших данных к использованию моделью:

1. Мы загружаем и выполняем пересемплирование аудиоданных на основе выборки, вызывая sample["audio"]. Библиотека 🤗 Datasets выполняет все необходимые операции пересемплирования на лету.
2. Мы используем извлекатель признаков для вычисления входных признаков в виде логарифмической мел-спектрограммы из нашего одномерного аудио-массива.
3. Мы кодируем транскрипции в идентификаторы меток с помощью токенизатора.

In [10]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=example["transcription"],
    )

    # вычисление длины входного аудиосэмпла в секундах
    example["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return example

Мы можем применить функцию подготовки данных ко всем нашим обучающим примерам, используя метод `.map` библиотеки 🤗 Datasets.
Мы удалим столбцы из исходных данных обучения (аудио и текст), оставив только столбцы, возвращаемые функцией `prepare_dataset`:

In [11]:
minds = minds.map(prepare_dataset, remove_columns=minds.column_names["train"], num_proc=1)

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

Далее мы фильтруем любые обучающие данные с аудиообразцами длиннее 30 секунд. В противном случае эти образцы могли бы быть усечены
экстрактором признаков Whisper, что может повлиять на стабильность обучения. Мы определяем функцию, которая возвращает `True` для образцов,
которые короче 30 секунд, и `False` для тех, что длиннее:

In [12]:
max_input_length = 30.0

def is_audio_in_length_range(length):
    return length < max_input_length

Теперь применяем нашу функцию фильтра ко всем образцам нашего набора обучающих данных с помощью метода `.filter` 🤗 Datasets:

In [13]:
minds["train"] = minds["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)

Filter:   0%|          | 0/450 [00:00<?, ? examples/s]

Мы можем сформировать пандас датафрейм из нашего датасета на текущем этапе, чтобы визуально оценить изменения.

Осторожно, выполнение ячейки ниже займет несколько гигабайт в оперативной памяти

In [14]:
import pandas as pd
df = pd.DataFrame(minds['train'])
df.head(5)

input_features  \
0  [[[-0.6715116500854492, -0.6283235549926758, -...   
1  [[[-0.23247992992401123, 0.47617435455322266, ...   
2  [[[-0.7375612258911133, -0.7375612258911133, -...   
3  [[[-0.5723974704742432, -0.5723974704742432, -...   
4  [[[-0.4922999143600464, -0.5557537078857422, -...   

                                              labels  input_length  
0  [50258, 50363, 40, 576, 411, 220, 1353, 992, 4...     10.837375  
1  [50258, 50363, 39, 268, 627, 6658, 992, 493, 2...      6.656000  
2  [50258, 50363, 4954, 286, 1116, 411, 220, 1353...     24.234625  
3  [50258, 50363, 4286, 360, 286, 722, 257, 7225,...      3.242625  
4  [50258, 50363, 7035, 291, 854, 385, 992, 493, ...      3.840000

In [15]:
df.shape

(445, 3)

Изначально наша обучаяющая выборка содержала 450 экземпляров, 5 из которых были отброшены в процессе фильтрации. В результате фактическое количество экземпляров обучающей выборки практически не изменилось, а это означает что подавляющее большинство аудиосемплов были длиной менее 30 секунд.

# Обучение и оценка

Для дообучения нам необходимо:

- Определить сборщика данных: сборщик данных берет наши предварительно обработанные данные и готовит тензоры PyTorch, готовые для модели.

- Определить метрику оценки: во время оценки мы хотим оценивать модель с использованием метрики Word Error Rate (WER). Нам нужно определить функцию `compute_metrics`, которая будет обрабатывать этот расчет.

- Загрузить предварительно обученную контрольную точку: нам нужно загрузить предварительно обученную контрольную точку и правильно сконфигурировать её для обучения.

- Определить аргументы обучения: они будут использоваться 🤗 Trainer для построения расписания обучения.


## Определение сборщика данных

Сборщик данных для модели речевой последовательности-в-последовательность уникален в том смысле, что он обрабатывает `input_features` и
`labels` независимо: `input_features` должны обрабатываться извлекателем признаков, а `labels` - токенизатором.

`input_features` уже дополнены по 30 секунд и преобразованы в логарифмическую мел-спектрограмму фиксированной размерности, поэтому все,
что нам нужно сделать, это преобразовать их в пакетированные тензоры PyTorch (батчи). Мы делаем это с помощью метода `.pad` экстрактора признаков
с параметром `return_tensors=pt`. Обратите внимание, что дополнительная подгонка не применяется, так как входные данные имеют
фиксированную размерность, и `input_features` просто преобразуются в тензоры PyTorch.

С другой стороны, `labels` не дополняются. Сначала мы дополняем последовательности до максимальной длины в пакете с использованием метода
`.pad` токенизатора. Затем дополнительные токены заполняются значением `-100`, чтобы эти токены **не** учитывались при вычислении потерь.
Затем мы удаляем начальный токен транскрипции из начала последовательности меток, так как мы добавим его позже во время обучения.

Мы можем воспользоваться ранее определенным `WhisperProcessor`, чтобы выполнить как операции извлекателя признаков, так и токенизатора:

In [16]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # разделение входов и меток, поскольку они должны иметь разную длину и требуют разных способов заполнения
        # сначала обрабатываем аудиовходы, просто возвращая pytorch тензоры
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # получить токенизированные последовательности меток
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # довести длину меток до максимального значения
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # замените padding на -100, чтобы корректно игнорировать потери
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # если токен bos был добавлен на предыдущем шаге токенизации,
        # вырезать токен bos здесь, так как он все равно будет добавлен позже
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Инициализируем только что определенный сборщик данных:

In [17]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

## Метрики оценки

При оценке систем распознавания речи мы сравниваем предсказания системы с транскрипцией целевого текста, аннотируя все имеющиеся ошибки.
Мы относим эти ошибки к одной из трех категорий:
1. Замены (S - от англ. "Substitutions"): когда мы транскрибируем **неправильное слово** в нашем предсказании ("sit" вместо "sat")
2. Вставки (I - от англ. "Insertions"): когда мы добавляем **дополнительное слово** в наше предсказание
3. Удаления (D - от англ. "Deletions"): когда мы **удаляем слово** в нашем предсказании


Метрика *word error rate (WER)* является "фактической" метрикой для распознавания речи. Она рассчитывает замены, вставки и удаления
на *уровне слова*. Это означает, что ошибки аннотируются на уровне каждого слова. Возьмем наш пример:

| Эталон:     | the | cat | sat     | on  | the | mat |
|-------------|-----|-----|---------|-----|-----|-----|
| Предсказание: | the | cat | **sit** | on  | the |     |  |
| Метка:      | ✅   | ✅   | S       | ✅   | ✅   | D   |

Здесь:
* 1 замена (S) ("sit" вместо "sat")
* 0 вставок
* 1 удаление (D) ("mat" отсутствует)

Это дает 2 ошибки в сумме. Чтобы получить коэффициент ошибок, разделим количество ошибок на общее количество слов в эталонной
последовательности (N), которое для данного примера равно 6:

$$
\begin{aligned}
WER &= \frac{S + I + D}{N} \\
&= \frac{1 + 0 + 1}{6} \\
&= 0.333
\end{aligned}
$$

Итак, WER равен 0,333, или 33,3%. Обратите внимание, что в слове "sit" ошибочным является только один символ,
но все слово помечено как неправильное. Это является отличительной особенностью WER: орфографические ошибки сильно штрафуются,
какими бы незначительными они ни были.

WER определяется так: чем меньше WER, тем меньше ошибок в прогнозе, поэтому для идеальной системы распознавания речи WER
был бы равен нулю (отсутствие ошибок).

Так как WER - это отношение количества ошибок к количеству слов (N), то верхнего предела для WER не существует!
Возьмем пример, когда мы предсказали 10 слов, а у целевой фразы только 2 слова. Если бы все наши прогнозы оказались неверными
(10 ошибок), то WER был бы равен 10 / 2 = 5, или 500%! Об этом следует помнить, если вы обучаете ASR-систему и видите, что WER
превышает 100%.

In [18]:
import evaluate

metric = evaluate.load("wer")

Нам нужно определить функцию, которая принимает предсказания нашей модели и возвращает метрику WER. Эта функция,
называемая `compute_metrics`, сначала заменяет `-100` на `pad_token_id` в `label_ids` (отменяя шаг, который мы применили в
сборщике данных, чтобы правильно игнорировать заполненные токены при вычислении потерь). Затем он декодирует предсказанные
идентификаторы и идентификаторы меток в строки. Наконец, она вычисляет WER между предсказаниями и эталонными метками.
Здесь у нас есть возможность оценить "нормализованные" транскрипции и предсказания, у которых удалены знаки препинания и регистр.

In [19]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # заменить -100 на pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # мы не хотим группировать токены при вычислении метрик
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # вычисление ортографического WER
    wer_ortho = metric.compute(predictions=pred_str, references=label_str)

    # вычисление нормированного WER
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    # шаг фильтрации для оценки только тех образцов, которые соответствуют ненулевым ссылкам:
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}

## Загрузка контрольной точки предобученной модели

In [20]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

Мы установим `use_cache` в значение `False` для обучения, так как мы используем [Gradient Checkpointing](https://huggingface.co/docs/transformers/v4.18.0/en/performance#gradient-checkpointing),
и эти две опции несовместимы.

Даже когда мы устанавливаем `batch_size` равным 1 и используем накопление градиента, нам все равно может не хватить памяти при работе с большими моделями. Для расчета градиентов во время обратного прохода все активации прямого прохода обычно сохраняются. Это может привести к большим затратам памяти. В качестве альтернативы можно забыть обо всех активациях во время прямого прохода и пересчитать их по требованию во время обратного прохода. Однако это приведет к значительным вычислительным затратам и замедлит обучение.

Gradient Checkpointing обеспечивает компромисс между двумя подходами и сохраняет стратегически выбранные активации по всему вычислительному графу, поэтому для градиентов необходимо повторно вычислять только часть активаций. 

Мы также переопределим два аргумента для генерации, чтобы контролировать поведение модели во время вывода:
мы принудительно зададим токены языка и задачи во время генерации, установив аргументы `language` и `task`, и также снова включим
кэш для генерации, чтобы ускорить время вывода:

In [21]:
from functools import partial

# отключить кэш во время обучения
model.config.use_cache = False
model.generate = partial(model.generate, use_cache=True)

## Определение конфигурации обучения

На последнем этапе мы определяем все параметры, связанные с обучением. Здесь мы устанавливаем количество шагов обучения на 500.
Этого количества шагов достаточно, чтобы увидеть большое улучшение WER по сравнению с предварительно обученной моделью Whisper,
при этом обеспечивая возможность выполнения дообучения в течение примерно 45 минут в бесплатном тарифе Google Colab.

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-tiny-finetuned-minds14",  # имя в HF Hub
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # увеличьте в 2 раза при уменьшении батча в 2 раза
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=500,  # можно увеличить исходя из того на каком графическом ускорителе планируете обучать
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

Мы можем передать аргументы обучения в 🤗 Trainer вместе с нашей моделью, набором данных, сборщиком данных и функцией `compute_metrics`:

In [23]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=minds["train"],
    eval_dataset=minds["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

## Обучение

In [24]:
trainer.train()

TrainOutput(global_step=500, training_loss=0.4329414145089686, metrics={'train_runtime': 1609.4358, 'train_samples_per_second': 4.971, 'train_steps_per_second': 0.311, 'total_flos': 1.9569551781888e+17, 'train_loss': 0.4329414145089686, 'epoch': 17.86})

In [26]:
tokenizer = processor
kwargs = {
    "dataset_tags": "PolyAI/minds14",
    "dataset": "minds14",
    "model_name": "whisper-tiny-finetuned-minds14",
    "finetuned_from": "openai/whisper-tiny",
    "tasks": "automatic-speech-recognition",
}

trainer.push_to_hub(**kwargs)
tokenizer.push_to_hub("whisper-tiny-finetuned-minds14")

CommitInfo(commit_url='https://huggingface.co/Lightmourne/whisper-tiny-finetuned-minds14/commit/2be914e369bd3ae074af45a58b4a7d118a2f0c91', commit_message='Upload processor', commit_description='', oid='2be914e369bd3ae074af45a58b4a7d118a2f0c91', pr_url=None, pr_revision=None, pr_num=None)

# Создание Демо с Gradio

Теперь, когда мы настроили модель Whisper для распознавания речи на Дивехи, давайте продолжим и создадим демо с использованием [Gradio](https://gradio.app).

Первое, что нам нужно сделать, это загрузить дообученную модель, используя класс `pipeline()`

In [30]:
from transformers import pipeline

model_id = "Lightmourne/whisper-tiny-finetuned-minds14"  # укажите id вашей контрольной точки
pipe = pipeline("automatic-speech-recognition", model=model_id)

Во-вторых, мы определим функцию, которая принимает путь к аудиофайлу в качестве входных данных и передает его через конвейер (`pipeline`).
Здесь конвейер автоматически заботится о загрузке аудиофайла, пересэмплировании его до правильной частоты дискретизации и выполнении вывода модели.
Затем мы просто вернем преобразованный текст в качестве выходных данных функции. Чтобы обеспечить возможность нашей модели обрабатывать аудиофайлы
произвольной длины, мы включим *фрагментирование* (англ. *chunking*) (разбиение на фрагменты).

In [31]:
def transcribe_speech(filepath):
    output = pipe(
        filepath,
        max_new_tokens=256,
        generate_kwargs={
            "task": "transcribe",
        },
        chunk_length_s=30,
        batch_size=8,
    )
    return output["text"]

Мы будем использовать функцию [blocks](https://gradio.app/docs/#blocks) в Gradio, чтобы создать две вкладки в нашем демо: одну
для транскрипции с микрофона и другую для загрузки файла.

In [37]:
import gradio as gr

demo = gr.Blocks()

mic_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(source="microphone", type="filepath"),
    outputs=gr.outputs.Textbox(),
)

file_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(source="upload", type="filepath"),
    outputs=gr.outputs.Textbox(),
)

<ipython-input-37-3a7fc6ce73ca>:8: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Textbox(),
<ipython-input-37-3a7fc6ce73ca>:14: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Textbox(),


Запускаем демо

In [ ]:
with demo:
    gr.TabbedInterface(
        [mic_transcribe, file_transcribe],
        ["Transcribe Microphone", "Transcribe Audio File"],
    )

demo.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:954: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

# Заключение

Мы рассмотрели пошаговое руководство по дообучению [”openai/whisper-tiny”](https://huggingface.co/openai/whisper-tiny) с использованием подмножества («en-US») датасета [”PolyAI/minds14”](https://huggingface.co/datasets/PolyAI/minds14) для решения задачи ASR используя 🤗 Datasets, Transformers и хаб Hugging Face.

Для оценки качества работы модели использовали метрику WER и достигли необходимого в задании baseline.

Дополнительную информацию по работе со звуком с использованием библиотек от Hugging Face вы можете найти в [аудио-курсе от Hugging Face - audio-transformers-course](https://github.com/huggingface/audio-transformers-course)